# ResNet 과제
## 1. 기본 코드(cifar10 dataset) 돌리면서 코드 파악

## 2. 원하는 dataset 선택
- dataset에 따른 imgsize & class 개수 등 조절 필요

## 3. Parameter 바꿔가면서 성능 변화 확인하기
  - image 크기(resize)
  - learning rate
  - optimizer

<br/>
<br/>

학습시간이 너무 오래 걸리면 epoch수를 줄이고, 다양한 시도를 해보면 좋을 것!!

참고 데이터셋
- https://www.tensorflow.org/api_docs/python/tf/keras/datasets
- 그 외 원하는 이미지 데이터셋 자유롭게 사용 가능

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
import tensorflow as tensorflow
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## ResNet 구현
### 1. identity block을 생성하는 함수인 identity_block() 생성. 
* input_tensor : 입력 tensor

* kernel_size : kernel 크기
  - identity block 내에 있는 두개의 conv layer중 1x1 kernel이 아니고, 3x3 kernel임 (3x3 커널이 아니라 5x5 kernel도 지정할 수 있게 구성)
* filters: 3개 conv layer들의 filter개수를 list 형태로 입력 받음
  * 첫번째 : 첫번째 1x1 filter 개수 (tensor의 channel 차원을 1/4로 축소)
  * 두번째 : 3x3 filter 개수
  * 세번째 : 마지막 1x1 filter 개수 (tensor의 차원 복구)
* stage: identity block들을 구별하기 위해서 설정 & 동일한 filter수를 가지는 identity block들은 동일한 stage로 설정
* block: 동일 stage내에서 identity block을 구별하기 위한 구분자 (a,b,c)

![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/residual_block_small.png)


In [ ]:
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation
from tensorflow.keras.layers import add, Add

# identity block은 shortcut 단에 conv layer가 없는 block 영역
def identity_block(input_tensor, middle_kernel_size, filters, stage, block):
    '''
    함수 입력 인자 설명
    input_tensor : 입력 tensor

    middle_kernel_size : 중간에 위치하는 kernel 크기. identity block내에 있는 두개의 conv layer중 1x1 kernel이 아니고, 3x3 kernel임. 
    (3x3 커널이 이외에도 5x5 kernel도 지정할 수 있게 구성)

    filters: 3개 conv layer들의 filter개수를 list 형태로 입력 받음. 첫번째 원소는 첫번째 1x1 filter 개수, 두번째는 3x3 filter 개수, 세번째는 마지막 1x1 filter 개수

    stage: identity block들이 여러개가 결합되므로 이를 구분하기 위해서 설정. 동일한 filter수를 가지는 identity block들을  동일한 stage로 설정.  

    block: 동일 stage내에서 identity block을 구별하기 위한 구분자
    ''' 
    
    # filters : filter 개수를 각각 filter1, filter2, filter3 list 형태로 할당. 
    # filter은 첫번째 1x1 filter 개수
    # filter2는 3x3 filter 개수
    # filter3는 마지막 1x1 filter 개수
    filter1, filter2, filter3 = filters
    
    # conv layer와 Batch normalization layer각각에 고유한 이름을 부여하기 위해 설정
    # 입력받은 stage와 block에 기반하여 이름 부여
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 이전 layer에 입력 받은 input_tensor를 기반으로 첫번째 1x1 Conv->Batch Norm->Relu 수행. 
    # 첫번째 1x1 Conv에서 Channel Dimension Reduction(1/4) 수행
    x = Conv2D(filters=filter1, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    x = Activation('relu')(x)
    
    # 두번째 3x3 Conv->Batch Norm->ReLU 수행
    # 3x3이 아닌 다른 kernel size도 구성 가능할 수 있도록 identity_block() 인자로 입력받은 middle_kernel_size를 이용
    # Conv 수행 출력 사이즈가 변하지 않도록 padding='same'으로 설정
    # filter 개수는 이전의 1x1 filter개수와 동일
    x = Conv2D(filters=filter2, kernel_size=middle_kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    # 마지막 1x1 Conv->Batch Norm 수행
    # ReLU를 수행 X (input tensor 더한 이후에 ReLU 적용)
    # filter 크기는 input_tensor channel 차원 개수로 복구
    x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2c')(x)

    # Residual Block 수행 결과 & input_tensor를 합 (Skip Connection)
    x = Add()([input_tensor, x])

    # 최종 ReLU 적용
    x = Activation('relu')(x)
    
    return x

### 2. 위에서 생성한 identity_block()을 호출하여 어떻게 identity block이 구성되어 있는지 확인

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

# input_tensor로 임의의 Feature Map size를 생성. 
input_tensor = Input(shape=(56, 56, 256), name='test_input')

# input_tensor의 channel수는 256개
# filters는 256의 1/4 filter수로 차원 축소후 다시 마지막 1x1 Conv에서 256으로 복원
filters = [64, 64, 256]

# 중간 Conv 커널 크기 : 3x3
kernel_size = (3, 3)
stage = 2
block = 'a'

# identity_block 호출 & layer들이 어떻게 구성되어 있는지 확인하기 위해서 model로 구성하고 summary()호출 
output = identity_block(input_tensor, kernel_size, filters, stage, block)
identity_layers = Model(inputs=input_tensor, outputs=output)
identity_layers.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 test_input (InputLayer)        [(None, 56, 56, 256  0           []                               
                                )]                                                                
                                                                                                  
 res2a_branch2a (Conv2D)        (None, 56, 56, 64)   16448       ['test_input[0][0]']             
                                                                                                  
 bn2a_branch2a (BatchNormalizat  (None, 56, 56, 64)  256         ['res2a_branch2a[0][0]']         
 ion)                                                                                             
                                                                                              

### 3. identity block을 연속으로 이어서 하나의 Stage 구성.
* 아래는 input tensor의 크기가 feature map 생성시 절반으로 줄지 않음
* input tensor의 크기가 절반으로 줄수 있도록 구성
* 동일한 Stage 내에서 feature map의 크기는 그대로 & block내에서 filter 개수는 변화

In [ ]:
input_tensor = Input(shape=(56, 56, 256), name='test_input')

x = identity_block(input_tensor, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='a')
x = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='b')

output = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='c')

identity_layers = Model(inputs=input_tensor, outputs=output)
identity_layers.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 test_input (InputLayer)        [(None, 56, 56, 256  0           []                               
                                )]                                                                
                                                                                                  
 res2a_branch2a (Conv2D)        (None, 56, 56, 64)   16448       ['test_input[0][0]']             
                                                                                                  
 bn2a_branch2a (BatchNormalizat  (None, 56, 56, 64)  256         ['res2a_branch2a[0][0]']         
 ion)                                                                                             
                                                                                            

### 4. 각 stage내의 첫번째 identity block에서 입력 feature map의 크기를 절반으로 줄이는 conv_block() 만들기
* conv_block() 함수는 앞에서 구현한 identity_block()함수과 거의 유사
* 입력 feature map의 크기를 절반으로 줄이고 shortcut 전달시 1x1 conv & stride 2 적용
* 첫번째 Stage의 첫번째 block에서는 이미 입력 feature map이 max pool로 절반이 줄어있는 상태이므로 다시 줄이지 않음

In [ ]:
def conv_block(input_tensor, middle_kernel_size, filters, stage, block, strides=(2, 2)):
    '''
    함수 입력 인자 설명
    input_tensor: 입력 tensor

    middle_kernel_size: 중간에 위치하는 kernel 크기. identity block내에 있는 두개의 conv layer중 1x1 kernel이 아니고, 3x3 kernel임. 
                        3x3 커널 이외에도 5x5 kernel도 지정할 수 있게 구성. 

    filters: 3개 conv layer들의 filter개수를 list 형태로 입력 받음. 첫번째 원소는 첫번째 1x1 filter 개수, 두번째는 3x3 filter 개수, 
             세번째는 마지막 1x1 filter 개수

    stage: identity block들이 여러개가 결합되므로 이를 구분하기 위해서 설정. 동일한 filter수를 가지는 identity block들을  동일한 stage로 설정.  

    block: 동일 stage내에서 identity block을 구별하기 위한 구분자

    strides: 입력 feature map의 크기를 절반으로 줄이기 위해서 사용. Default는 2이지만, 
             첫번째 Stage의 첫번째 block에서는 이미 입력 feature map이 max pool로 절반이 줄어있는 상태이므로 다시 줄이지 않기 위해 1을 호출해야함 
    ''' 
    
    # filters : filter 개수를 각각 filter1, filter2, filter3 list 형태로 할당. 
    # filter은 첫번째 1x1 filter 개수
    # filter2는 3x3 filter 개수
    # filter3는 마지막 1x1 filter 개수
    filter1, filter2, filter3 = filters

    # conv layer와 Batch normalization layer각각에 고유한 이름을 부여하기 위해 설정. 입력받은 stage와 block에 기반하여 이름 부여
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 이전 layer에 입력 받은 input_tensor를 기반으로 첫번째 1x1 Conv->Batch Norm->Relu 수행. 
    # 입력 feature map 사이즈를 1/2로 줄이기 위해 strides 입력  
    x = Conv2D(filters=filter1, kernel_size=(1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base+'2a')(input_tensor)
    
    # Batch Norm 적용
    # 입력 데이터는 batch 사이즈까지 포함하여 4차원 : (batch_size, height, width, channel depth)
    # Batch Norm의 axis는 channel depth에 해당하는 axis index인 3을 입력 (무조건 channel이 마지막 차원의 값으로 입력된다고 가정)
    x = BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    # ReLU Activation 적용
    x = Activation('relu')(x)
    
    # 두번째 3x3 Conv->Batch Norm->ReLU 수행
    # 3x3이 아닌 다른 kernel size도 구성 가능할 수 있도록 identity_block() 인자로 입력받은 middle_kernel_size를 이용. 
    # Conv 수행 출력 사이즈가 변하지 않도록 padding='same'으로 설정. filter 개수는 이전의 1x1 filter개수와 동일.  
    x = Conv2D(filters=filter2, kernel_size=middle_kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base+'2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = Activation('relu')(x)
    
    # 마지막 1x1 Conv->Batch Norm 수행
    # ReLU를 수행 X (input tensor 더한 이후에 ReLU 적용)
    # filter 크기는 input_tensor channel 차원 개수로 복구
    x = Conv2D(filters=filter3, kernel_size=(1, 1), kernel_initializer='he_normal', name=conv_name_base+'2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base+'2c')(x)
    
    # shortcut을 1x1 conv 수행, filter3가 입력 feature map의 filter 개수
    shortcut = Conv2D(filter3, (1, 1), strides=strides, kernel_initializer='he_normal', name=conv_name_base+'1')(input_tensor)
    shortcut = BatchNormalization(axis=3, name=bn_name_base+'1')(shortcut)
    
    # Residual Block 수행 결과 & 1x1 conv가 적용된 shortcut을 합 
    x = add([x, shortcut])
    
    # 최종 ReLU 적용
    x = Activation('relu')(x)
    
    return x
    

### 5. conv_block()과 identity_block()을 호출하여 stage 구성.

In [ ]:
input_tensor = Input(shape=(56, 56, 256), name='test_input')

# conv_block() 호출 시 strides를 2로 설정하여 입력 feature map의 크기를 절반으로 줄임 / strides=1이면 크기를 그대로 유지
x = conv_block(input_tensor, middle_kernel_size=3, filters=[64, 64, 256], strides=2, stage=2, block='a')
x = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='b')

output = identity_block(x, middle_kernel_size=3, filters=[64, 64, 256], stage=2, block='c')

identity_layers = Model(inputs=input_tensor, outputs=output)
identity_layers.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 test_input (InputLayer)        [(None, 56, 56, 256  0           []                               
                                )]                                                                
                                                                                                  
 res2a_branch2a (Conv2D)        (None, 28, 28, 64)   16448       ['test_input[0][0]']             
                                                                                                  
 bn2a_branch2a (BatchNormalizat  (None, 28, 28, 64)  256         ['res2a_branch2a[0][0]']         
 ion)                                                                                             
                                                                                            

### 6. input image를 7x7 Conv 변환하고 Max Pooling 적용 로직을 별도 함수로 구현.
* O = (I - F + 2P)/S + 1, I는 Input size, F는 filter의 kernel 크기, P는 padding, S는 Stride
* (224 - 7)/2 + 1 = 109.5 = 109가 됨. 따라서 112x112 로 출력하기 위해 ZeroPadding2D(3, 3)수행
* 112x112로 MaxPooling 을 (3, 3) pool size로 stride 2로 수행하므로 56x56으로 출력하기 위해 ZeroPadding2D(1,1) 수행

In [ ]:
from tensorflow.keras.layers import ZeroPadding2D, MaxPooling2D

def do_first_conv(input_tensor):
    # 7x7 Conv 연산 수행하여 feature map 생성, input_tensor 크기를 절반으로 생성
    # filter 개수 : 64개 
    # 224x224 를 input -> 7x7 conv, strides=2 -> 112x112 출력 (Zero padding 적용)
    x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(input_tensor)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid', kernel_initializer='he_normal', name='conv')(x)
    x = BatchNormalization(axis=3, name='bn_conv1')(x)
    x = Activation('relu')(x)

    # 다시 feature map 크기를 MaxPooling으로 절반으로 만듬 -> 56x56으로 출력 (zero padding 적용)
    x = ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
    return x

input_tensor = Input(shape=(224, 224, 3))
output = do_first_conv(input_tensor)
model = Model(inputs=input_tensor, outputs=output)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)       0         
                                                                 
 conv (Conv2D)               (None, 112, 112, 64)      9472      
                                                                 
 bn_conv1 (BatchNormalizatio  (None, 112, 112, 64)     256       
 n)                                                              
                                                                 
 activation_21 (Activation)  (None, 112, 112, 64)      0         
                                                                 
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)      0         
                                                           

### 7. ResNet 50 모델 생성.
* 앞에서 생성한 conv_block()과 identity_block()을 호출하여 ResNet 50 모델 생성. 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler

def create_resnet(in_shape=(224, 224, 3), n_classes=10):
    input_tensor = Input(shape=in_shape)
    
    # 첫번째 7x7 Conv와 Max Pooling 적용.  
    x = do_first_conv(input_tensor)
    
    # stage 2의 conv_block과 identity block 생성
    # stage2의 첫번째 conv_block은 strides를 1로 하여 크기를 줄이지 않음. 
    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')
    
    # stage 3의 conv_block과 identity block 생성
    # stage3의 첫번째 conv_block은 strides를 2(default)로 하여 크기를 줄임 
    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    # stage 4의 conv_block과 identity block 생성
    # stage4의 첫번째 conv_block은 strides를 2(default)로 하여 크기를 줄임
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    # stage 5의 conv_block과 identity block 생성
    # stage5의 첫번째 conv_block은 strides를 2(default)로 하여 크기를 줄임
    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    # classification dense layer와 연결 전 GlobalAveragePooling 수행 
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(200, activation='relu', name='fc_01')(x)
    x = Dropout(rate=0.5)(x)
   
    # 마지막 fully connected layer & Softmax 함수를 이용해 확률 반환
    output = Dense(n_classes, activation='softmax', name='fc_final')(x) 
    
    # model 구성
    model = Model(inputs=input_tensor, outputs=output, name='resnet50')
    model.summary()
    
    return model

In [ ]:
model =  create_resnet(in_shape=(224,224,3), n_classes=10)

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv (Conv2D)                  (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

## CIFAR10 데이터 세트로 ResNet 모델 학습 및 성능 테스트

In [ ]:
IMAGE_SIZE = 128
BATCH_SIZE = 64

### 데이터 전처리/인코딩/스케일링 함수 및 CIFAR_Dataset 선언

In [ ]:
import random as python_random
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import Sequence
import cv2
import sklearn

def zero_one_scaler(image):
    return image/255.0

# One Hot Encoding(OHE)
def get_preprocessed_ohe(images, labels, pre_func=None):
    # preprocessing 함수가 입력되면 이를 이용하여 image array를 scaling 적용.
    if pre_func is not None:
        images = pre_func(images)
    # OHE 적용    
    oh_labels = to_categorical(labels)
    return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=42):

    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경
    # Label에 대해서는 OHE 적용
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # train valid split
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels )

from tensorflow.keras.utils import Sequence
import cv2
import sklearn

# 입력 인자 images_array labels는 모두 numpy array로 들어옴 (images_array는 32x32)
class CIFAR_Dataset(Sequence):
    def __init__(self, images_array, labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=None):
        '''
        파라미터 설명

        images_array : 원본 32x32 만큼의 image 배열값. 

        labels : 해당 image의 label

        batch_size : __getitem__(self, index) 호출 시 마다 가져올 데이터 batch 건수

        augmentor : albumentations 객체

        shuffle : 학습 데이터의 경우 epoch 종료시마다 데이터를 섞을지 여부
        '''
        # 객체 생성 인자로 들어온 값을 객체 내부 변수로 할당. 
        # 인자로 입력되는 images_array는 전체 32x32 image array임.
        self.images_array = images_array
        self.labels = labels
        self.batch_size = batch_size
        self.augmentor = augmentor
        self.pre_func = pre_func

        # train data의 경우 
        self.shuffle = shuffle
        if self.shuffle:
            # 객체 생성시에 한번 데이터를 섞음. 
            #self.on_epoch_end()
            pass
    
    # Sequence를 상속받은 Dataset은 batch_size 단위로 입력된 데이터를 처리함. 
    # __len__()은 전체 데이터 건수가 주어졌을 때 batch_size단위로 몇번 데이터를 반환하는지 나타남
    def __len__(self):
        # batch_size단위로 데이터를 몇번 가져와야하는지 계산하기 위해 전체 데이터 건수를 batch_size로 나누되, 정수로 정확히 나눠지지 않을 경우 1회를 더한다. 
        return int(np.ceil(len(self.labels) / self.batch_size))
    
    # batch_size 단위로 image_array, label_array 데이터를 가져와서 변환 후 반환
    # index(몇번째 배치이지) -> 해당 순서에 해당하는 batch_size 만큼의 데이터를 가공하여 반환
    # batch_size 개수만큼 image_array와 label_array 반환
    def __getitem__(self, index):
        # batch_size만큼 순차적으로 데이터를 가져오려면 array에서 index*self.batch_size : (index+1)*self.batch_size 만큼 가져옴
        # 32x32 image array를 self.batch_size만큼 가져옴. 
        images_fetch = self.images_array[index*self.batch_size:(index+1)*self.batch_size]

        if self.labels is not None:
            label_batch = self.labels[index*self.batch_size:(index+1)*self.batch_size]
        
        # albumentation으로 만든 augmentor가 주어진다면 augmentor를 이용
        # albumentations : image만 변환할 수 있으므로 batch_size만큼 할당된 image_name_batch를 한 건씩 iteration하면서 변환 수행 
        # 변환된 image 배열값을 담을 image_batch 선언 & image_batch 배열은 float32 로 설정 
        image_batch = np.zeros((images_fetch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3), dtype='float32')
        
        # batch_size에 담긴 건수만큼 iteration 하면서 opencv image load -> image augmentation 변환
        # augmentor가 not None일 경우 -> image_batch에 담음
        for image_index in range(images_fetch.shape[0]):
            #image = cv2.cvtColor(cv2.imread(image_name_batch[image_index]), cv2.COLOR_BGR2RGB)
            
            # 원본 image를 IMAGE_SIZE x IMAGE_SIZE 크기로 변환
            image = cv2.resize(images_fetch[image_index], (IMAGE_SIZE, IMAGE_SIZE))

            # 만약 augmentor가 주어졌다면 이를 적용. 
            if self.augmentor is not None:
                image = self.augmentor(image=image)['image']
                
            # 만약 scaling 함수가 입력되었다면 이를 적용하여 scaling 수행. 
            if self.pre_func is not None:
                image = self.pre_func(image)
            
            # image_batch에 순차적으로 변환된 image를 담음.               
            image_batch[image_index] = image
        
        return image_batch, label_batch
    
    # epoch가 한번 수행이 완료 될 때마다 모델의 fit()에서 호출됨. 
    def on_epoch_end(self):
        if(self.shuffle):
            #print('epoch end')
            # 원본 image배열과 label를 쌍을 맞춰서 섞어준다. scikt learn의 utils.shuffle에서 해당 기능 제공
            self.images_array, self.labels = sklearn.utils.shuffle(self.images_array, self.labels)
        else:
            pass


### 원-핫 인코딩, 학습/검증/테스트 데이터 세트 분할
* scaling은 원본 채널별 pixel값 - [103.939, 116.779, 123.68] 적용.



In [ ]:
# CIFAR10 데이터 재 로딩 및 OHE 전처리 적용하여 학습/검증/데이터 세트 생성. 
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.2, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_oh_labels.shape)

169001437/169001437 [==============================] - 13s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)
(40000, 32, 32, 3) (40000, 100) (10000, 32, 32, 3) (10000, 100) (10000, 32, 32, 3) (10000, 100)


### 학습, 검증용 CIFAR_Dataset 생성

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

tr_ds = CIFAR_Dataset(tr_images, tr_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=True, pre_func=resnet_preprocess)
val_ds = CIFAR_Dataset(val_images, val_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)

print(next(iter(tr_ds))[0].shape, next(iter(val_ds))[0].shape)
print(next(iter(tr_ds))[1].shape, next(iter(val_ds))[1].shape)
# 채널별 값 - [103.939, 116.779, 123.68]
print(next(iter(tr_ds))[0][0])

(64, 128, 128, 3) (64, 128, 128, 3)
(64, 100) (64, 100)
[[[118.061     128.22101   128.32     ]
  [118.061     128.22101   128.32     ]
  [116.061     127.221     127.32     ]
  ...
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]]

 [[118.061     128.22101   128.32     ]
  [118.061     128.22101   128.32     ]
  [116.061     126.221     127.32     ]
  ...
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]]

 [[117.061     127.221     127.32     ]
  [117.061     127.221     127.32     ]
  [115.061     125.221     126.32     ]
  ...
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]
  [139.061     130.22101   118.32     ]]

 ...

 [[-23.939003  -49.779     -66.68     ]
  [-23.939003  -49.779     -66.68     ]
  [-24.939003  -50.779     -67.68     ]
  ...
  [-21.939003    9.221001   51.32     ]
  [-29.939003    3.2210007  

### 1) ResNet50 (모델 생성) 후 학습/평가
* 초기 learning_rate 0.001



In [ ]:
resnet_model = create_resnet(in_shape=(128, 128, 3), n_classes=100)

resnet_model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# 5번 iteration내에 validation loss가 향상되지 않으면 learning rate을 기존 learning rate * 0.2로 줄임.  
rlr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, mode='min', verbose=1)
ely_cb = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)

history = resnet_model.fit(tr_ds, epochs=10, 
                    #steps_per_epoch=int(np.ceil(tr_images.shape[0]/BATCH_SIZE)),
                    validation_data=val_ds, 
                    #validation_steps=int(np.ceil(val_images.shape[0]/BATCH_SIZE)), 
                    callbacks=[rlr_cb, ely_cb]
                   )

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv (Conv2D)                  (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 bn_conv1 (BatchNormalization)  (None, 64, 64, 64)   256         ['conv[0][0]']            

In [ ]:
test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)
resnet_model.evaluate(test_ds)

157/157 [==============================] - 12s 73ms/step - loss: 4.6056 - accuracy: 0.0100


[4.605584144592285, 0.009999999776482582]

### 2) ResNet50 (Pretrained 모델)로 학습/평가

In [ ]:
from tensorflow.keras.applications import ResNet50

input_tensor = Input(shape=(128, 128, 3))
base_model = ResNet50(include_top=False, weights=None, input_tensor=input_tensor)
bm_output = base_model.output

# classification dense layer와 연결 전 GlobalAveragePooling 수행 
x = GlobalAveragePooling2D(name='avg_pool')(bm_output)
x = Dropout(rate=0.5)(x)
x = Dense(200, activation='relu', name='fc_01')(x)
x = Dropout(rate=0.5)(x)
output = Dense(100, activation='softmax', name='fc_final')(x)

pr_model = Model(inputs=input_tensor, outputs=output, name='resnet50')
pr_model.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [ ]:
pr_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = pr_model.fit(tr_ds, epochs=40, 
                    validation_data=val_ds,
                    callbacks=[rlr_cb, ely_cb]
                   )

test_ds = CIFAR_Dataset(test_images, test_oh_labels, batch_size=BATCH_SIZE, augmentor=None, shuffle=False, pre_func=resnet_preprocess)
pr_model.evaluate(test_ds)

Epoch 1/40
625/625 [==============================] - 163s 253ms/step - loss: 3.0745 - accuracy: 0.2450 - val_loss: 2.8890 - val_accuracy: 0.2828 - lr: 1.0000e-04
Epoch 2/40
625/625 [==============================] - 157s 251ms/step - loss: 2.9071 - accuracy: 0.2743 - val_loss: 2.7830 - val_accuracy: 0.3066 - lr: 1.0000e-04
Epoch 3/40
625/625 [==============================] - 157s 251ms/step - loss: 2.7631 - accuracy: 0.3007 - val_loss: 2.7235 - val_accuracy: 0.3255 - lr: 1.0000e-04
Epoch 4/40
625/625 [==============================] - 157s 251ms/step - loss: 2.5685 - accuracy: 0.3397 - val_loss: 2.6092 - val_accuracy: 0.3416 - lr: 1.0000e-04
Epoch 5/40
625/625 [==============================] - 157s 251ms/step - loss: 2.3776 - accuracy: 0.3742 - val_loss: 2.6513 - val_accuracy: 0.3365 - lr: 1.0000e-04
Epoch 6/40
625/625 [==============================] - 157s 251ms/step - loss: 2.2249 - accuracy: 0.4089 - val_loss: 2.5796 - val_accuracy: 0.3614 - lr: 1.0000e-04
Epoch 7/40
625/625 [==

[3.214346408843994, 0.4341999888420105]

#Pytorch Image Classification

* tensorflow로 구현된 내용을 필자가 자주 활용하는 pytorch로 변환하여 과제 수행
* 전체 Resnet을 구현하지는 않고, torchvision에서 이미 구현된 resnet을 활용하여 과제 수행
* 나머지 사항들은 각주로서 설명

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms 

if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'
print(device)

cuda


* Pytorch identity_block/conv_block/first_conv 구현
     
     *사실 Pytorch 특성상 이렇게 구현하면 안되는데..(pytorch는 클래스 중심으로 구현) tensorflow의 block들을 직관적으로 구현하고 이해하고자 부득이하게 단일 메소드로 구현.. 

In [ ]:
from torch import nn

def identity_block(input_tensor, middle_kernel_size, filters):

  #def __init__():
  #self.input_tensor=input_tensor
  #self.middle_kernel_size=middle_kernel_size
  #self.filters=filters

  #def _make_layers(input_tensor, middle_kernel_size, filters):
  filter1, filter2, filter3=filters
  relu=nn.ReLU()
  conv1=nn.Conv2d(input_tensor.shape[1], filter1, kernel_size=(1,1))
  bnorm1=nn.BatchNorm2d(filter1)

  conv2=nn.Conv2d(filter1, filter2, kernel_size=middle_kernel_size)
  bnorm2=nn.BatchNorm2d(filter2)

  conv3=nn.Conv2d(filter2, filter3, kernel_size=(1,1))
  bnorm3=nn.BatchNorm2d(filter3)

  #def forward():
  x=conv1(input_tensor)
  x=bnorm1(x)
  x=relu(x)
  x=conv2(x)
  x=bnorm2(x)
  x=relu(x)
  x=conv3(x)
  x=bnorm3(x)
  
  x=torch.add(input_tensor, x)
  x=relu(x)

  return x

In [ ]:
def conv_block(input_tensor, middle_kernel_size, filters, strides=(2,2)):


  #def _make_layers(input_tensor, middle_kernel_size, filters, strides):
  filter1, filter2, filter3=filter1

  relu=nn.ReLU()

  conv1=nn.Conv2d(input_tensor.shape[1], filter1, kernel_size=(1,1), stride=2)
  bnorm1=nn.BatchNorm2d(filter1)

  conv2=nn.Conv2d(filter1, filter2, kernel_size=middle_kernel_size, padding=1)
  bnorm2=nn.BatchNorm2d(filter2)

  conv3=nn.Conv2d(filter2, filter3, kernel_size=(1,1))
  bnorm3=nn.BatchNorm2d(filter3)

  shortcut1=nn.Conv2d(filter3, filter3, kernel_size=(1,1), stride=strides)
  shortcut2=nn.BatchNorm2d(filter3)

  #def forward():
  x=conv1(input_tensor)
  x=bnorm1(x)
  x=relu(x)
  x=conv2(x)
  x=bnorm2(x)
  x=relu(x)
  x=conv3(x)
  x=bnorm3(x)
  
  shortcut=shortcut1(input_tensor)
  shortcut=shortcut2(shortcut)

  x=torch.add(x, shortcut)
  x=relu(x)

  return x

In [ ]:
def first_conv(input_tensor):

  #def _make_layers(input_tensor):
  pad1=nn.ZeroPad2d((3,3))
  conv1=nn.Conv2d(64, (7,7),strides=(2,2))
  bnorm1=nn.BatchNorm2d(64)
  relu=nn.ReLU()

  pad2=nn.ZeroPad2d((1,1))
  maxpool=nn.MaxPool2d((3,3),strides=(2,2))

  #def forward():
  x=pad1(input_tensor)
  x=conv1(x)
  x=bnorm1(x)
  x=relu(x)

  x=pad2(x)
  x=maxpool(x)

  return x

#Pytorch Resnet

In [ ]:
tr_data=datasets.CIFAR100(root='data',
                          train=True,
                          download=True,
                          )


Files already downloaded and verified


In [ ]:
#Image Normalization
x = np.concatenate([np.asarray(tr_data[i][0]) for i in range(len(tr_data))])

mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0, 1))/255

mean=mean.tolist()
std=std.tolist()

In [ ]:
tr_data=datasets.CIFAR100(root='data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([ #Simple Augmentation 적용
                              transforms.Resize(224), #원래 224*224 이미지에 적합한 모델이므로 Resize
                              transforms.RandomCrop(224, padding=4,padding_mode='reflect'), #224*224 image에서 image RandomCrop(*image augmentation)
                              transforms.RandomHorizontalFlip(), #horizontal 방향으로 사진 뒤집기
                              transforms.ToTensor(),
                              transforms.Normalize(mean=mean,
                                     std=std,inplace=True)
                          ]))

test_data=datasets.CIFAR100(root='data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                              transforms.Resize(224),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=mean,
                                     std=std)
                          ]))

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
batch_size= 32

train_db=DataLoader(tr_data, batch_size=batch_size,shuffle=True,pin_memory=True)
test_db=DataLoader(test_data, batch_size=batch_size,shuffle=False,pin_memory=True)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=False) #Resnet 호출 
model.to(device) #cuda 올리기

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=5e-3,weight_decay=1e-5)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
from torchsummary import summary #model summary check

summary(model, (3,224,224), batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
            Conv2d-5           [32, 64, 56, 56]           4,096
       BatchNorm2d-6           [32, 64, 56, 56]             128
              ReLU-7           [32, 64, 56, 56]               0
            Conv2d-8           [32, 64, 56, 56]          36,864
       BatchNorm2d-9           [32, 64, 56, 56]             128
             ReLU-10           [32, 64, 56, 56]               0
           Conv2d-11          [32, 256, 56, 56]          16,384
      BatchNorm2d-12          [32, 256, 56, 56]             512
           Conv2d-13          [32, 256, 56, 56]          16,384
      BatchNorm2d-14          [32, 256,

In [ ]:
#마지막 Linear layer의 softmax가 1000-d로 구성되어 있음 _ ImageNet Classification에 적합한 모델
#Cifar-100은 100개의 label이기 때문에 100-d로 linear layer 수정
for name, child in model.named_children():
    if isinstance(child, nn.Linear):
        model._modules[name]=nn.Linear(2048, 100).to(device) #instance 변경 후 device에 올리기
    elif isinstance(child, nn.Sequential): #sequential 안에 있으면 거기에서 변환
        for sname, schild in child.named_children():
            if isinstance(schild, nn.Linear):
                print(name,sname)
                model._modules[name]._modules[sname]=nn.Linear(2048, 100).to(device)

In [ ]:
summary(model, (3,224,224), batch_size=32) #100-d로 변환 확인

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 112, 112]           9,408
       BatchNorm2d-2         [32, 64, 112, 112]             128
              ReLU-3         [32, 64, 112, 112]               0
         MaxPool2d-4           [32, 64, 56, 56]               0
            Conv2d-5           [32, 64, 56, 56]           4,096
       BatchNorm2d-6           [32, 64, 56, 56]             128
              ReLU-7           [32, 64, 56, 56]               0
            Conv2d-8           [32, 64, 56, 56]          36,864
       BatchNorm2d-9           [32, 64, 56, 56]             128
             ReLU-10           [32, 64, 56, 56]               0
           Conv2d-11          [32, 256, 56, 56]          16,384
      BatchNorm2d-12          [32, 256, 56, 56]             512
           Conv2d-13          [32, 256, 56, 56]          16,384
      BatchNorm2d-14          [32, 256,

In [ ]:
def train(dataloader, model, loss_fn, optimizer,epochs,max_lr,grad_clip=None):
  size = len(dataloader.dataset)
  sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(dataloader))
  lrs=[]
  model.train()
  for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        if grad_clip: #gradient clipping
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lrs.append(get_lr(optimizer))
        sched.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(loss)

def test(dataloader, model, loss_fn):
  
  size=len(dataloader.dataset)
  num_batches=len(dataloader)
  model.eval()
  loss, correct=0,0
  with torch.no_grad():
    for X, y in dataloader:
      X, y=X.to(device), y.to(device)
      pred=model(X)
      loss+=loss_fn(pred,y).item()
      correct+=(pred.argmax(1)==y).type(torch.float).sum().item()
  loss/=num_batches
  correct/=size
  print('accuracy:'+str(100*correct), 'loss:' +str(loss))

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
epoch = 20 #20 Epoch에 정확도 61% 수준,, 
for t in range(epoch):

  print('epoch('+str(t+1)+'/'+str(epoch)+')')
  train(train_db, model, loss_fn, optimizer,epoch, 0.01)
  test(test_db, model, loss_fn)
  
print('done!')

epoch(1/20)
4.615495681762695
4.376228332519531
3.343193292617798
3.0038888454437256
3.0677719116210938
2.8942923545837402
2.7303531169891357
3.1208972930908203
3.1054129600524902
2.762056350708008
3.0382602214813232
3.472717761993408
2.252007007598877
2.7657196521759033
2.933149576187134
3.0089261531829834
accuracy:36.78 loss:2.5490293632300136
epoch(2/20)
2.3823137283325195
2.423600435256958
2.208503246307373
2.263887882232666
2.4532291889190674
1.533050775527954
2.629459857940674
2.5157315731048584
2.5312321186065674
2.559492588043213
1.6848491430282593
2.600003719329834
2.5988986492156982
2.4901270866394043
3.0182278156280518
2.290137767791748
accuracy:39.54 loss:2.439271952016666
epoch(3/20)
2.224228858947754
2.2281625270843506
2.099073648452759
2.201167106628418
2.2201578617095947
1.859486699104309
1.9062190055847168
1.7283892631530762
1.8501701354980469
2.7545695304870605
2.237600564956665
2.3745172023773193
2.3602795600891113
2.871743679046631
3.0025010108947754
1.9429994821548